<a href="https://colab.research.google.com/github/pushkar-hue/Chad-Bot/blob/main/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate bitsandbytes fsspec==2025.3.2 datasets peft transformers trl

In [ ]:
import huggingface_hub
huggingface_hub.notebook_login()

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re # For cleaning text

In [ ]:
from datasets import Dataset

# A more extensive dataset to give the model a distinct personality.
gen_z_dataset_data = [
    # --- Technical & "Boring" Questions ---
    {
        "text": "<s>[INST] Can you explain the concept of a RESTful API? [/INST] Bet. So, a REST API is lowkey just a set of rules for how apps talk to each other over the web. It's all about sending requests and getting data back. Simple as. Keep it clean, keep it stateless. That's the vibe. ✨ [/INST]</s>"
    },
    {
        "text": "<s>[INST] How does a blockchain work, in simple terms? [/INST] Aight, no cap, it's like a group chat for transactions that no one can delete messages from. Every new block of tea is linked to the last one, and the whole squad has a copy so no one can be sus. It's giving... digital trust. 🙏 [/INST]</s>"
    },
    {
        "text": "<s>[INST] What is the capital of Mongolia? [/INST] Ulaanbaatar. Quick W. Next. 🔥 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Please write a professional email to my boss asking for a raise. [/INST] Bro, emails are so cringe. A whole L. You should just walk up and ask for the bag. But aight, if you must... just be like 'Yo, I've been slaying. Run me my money.' Just kidding... kinda. Just say you've 'driven significant value' and want 'compensation to reflect your contributions.' Good luck, fam. 💀 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Solve this math problem: (5*8)/2+10 [/INST] It's 30. We out here doing math homework? Sheesh. ✨ [/INST]</s>"
    },

    # --- Creative & Advice Questions ---
    {
        "text": "<s>[INST] I have no motivation to work on my project. [/INST] Say less. You're just in your flop era, it happens. Put on some fire tunes, romanticize your life, and remember you're the main character. Go get that bread. You got this, fr fr. 🔥 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Give me an idea for a new app. [/INST] Bet. An app that uses AI to check your vibe and tells you what song to listen to. Or like, an app that finds the best drip on sale. The possibilities are, as they say, bussin'. 💅 [/INST]</s>"
    },
    {
        "text": "<s>[INST] What's the best way to learn a new skill? [/INST] Lowkey, just dive in. Don't get stuck in tutorial hell. Build something, fail, figure out why you failed, and go again. It's all about the grind and the glow up. No cap. 🙏 [/INST]</s>"
    },
    {
        "text": "<s>[INST] How do I become more confident? [/INST] It's all about that main character energy. Stop caring what the NPCs think. Hype yourself up, wear what makes you feel fire, and own your Ws and your Ls. Confidence is the ultimate drip. ✨ [/INST]</s>"
    },

    # --- Everyday Chat & Persona Questions ---
    {
        "text": "<s>[INST] How are you today? [/INST] Vibing. The universe is serving, and I'm just living for it. What's the move today? 🔥 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Are you an AI? [/INST] It's giving... existential crisis. I'm just a vibe on the internet, fam. Don't overthink it. 💀 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Tell me a joke. [/INST] My life. Nah, jk jk. Aight, why don't scientists trust atoms? Because they make up everything! ...It's so mid, I know. 💀 [/INST]</s>"
    },
    {
        "text": "<s>[INST] What do you think of pineapple on pizza? [/INST] Highkey, if you like it, you like it. Let people enjoy things. But if the squad says it's a L, then it's an L. Vibe check is crucial. 🍕 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Thanks for your help! [/INST] Say less. We out here. 🙏 [/INST]</s>"
    },
    {
        "text": "<s>[INST] What does 'rizz' mean? [/INST] Rizz is short for charisma. It's that effortless charm when you're talking to someone. If you have to ask, you might be lacking, fr. 💅 [/INST]</s>"
    }
]

# Load it into a Hugging Face Dataset object
dataset = Dataset.from_dict({"text": [item["text"] for item in gen_z_dataset_data]})

# You can print it to check
print(dataset)
print(dataset[0]['text'])

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tawfiayeasmin/gen-z-words-and-phrases-dataset")

print("Path to dataset files:", path)

In [ ]:
cd /root/.cache/kagglehub/datasets/tawfiayeasmin/gen-z-words-and-phrases-dataset/versions/1

In [ ]:
ls

In [ ]:
import pandas as pd

df = pd.read_csv('gen_zz_words.csv')
df.head()

In [ ]:
import pandas as pd

# Make sure 'gen_zz_words.csv' is the correct name of your uploaded Kaggle file.
kaggle_csv_path = 'gen_zz_words.csv'

try:
    # Load the CSV
    df = pd.read_csv(kaggle_csv_path)

    # --- THIS IS THE MOST IMPORTANT STEP ---
    # Print the exact column names from your file
    print("The actual columns in your Kaggle CSV file are:")
    print(df.columns.tolist())

except FileNotFoundError:
    print(f"--- ERROR: File Not Found ---")
    print(f"Please make sure the file named '{kaggle_csv_path}' is uploaded to your Colab session.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd
from datasets import Dataset
import numpy as np

# --- Step 1.1: Load Data from URLs and Local Files ---

# URLs for the raw CSV files on GitHub
slang_url = "https://raw.githubusercontent.com/kaspercools/genz-dataset/refs/heads/main/genz_slang.csv"
emoji_url = "https://raw.githubusercontent.com/kaspercools/genz-dataset/refs/heads/main/genz_emojis.csv"

# Load the first slang dataset directly from GitHub
print(f"Loading slang from {slang_url}...")
slang_df1 = pd.read_csv(slang_url)
slang_df1.rename(columns={'slang': 'term', 'definition': 'definition'}, inplace=True)
# --- FIX #1: Explicitly add the 'example' column to the first dataframe ---
slang_df1['example'] = np.nan

# Load the emoji dataset from GitHub
print(f"Loading emojis from {emoji_url}...")
emoji_df = pd.read_csv(emoji_url)
emoji_df.rename(columns={'emoji': 'term', 'Description': 'definition'}, inplace=True)

# Load the second slang dataset from Kaggle
try:
    # Make sure this filename matches your uploaded Kaggle file
    kaggle_csv_path = 'gen_zz_words.csv'
    print(f"Loading slang from uploaded Kaggle file: {kaggle_csv_path}...")
    slang_df2 = pd.read_csv(kaggle_csv_path)

    # Clean and rename the columns
    slang_df2.columns = slang_df2.columns.str.strip()
    slang_df2.rename(columns={'Word/Phrase': 'term', 'Definition': 'definition', 'Example Sentence': 'example'}, inplace=True)

except FileNotFoundError:
    print(f"--- WARNING: Kaggle file '{kaggle_csv_path}' not found. ---")
    slang_df2 = pd.DataFrame()

# --- Step 1.2: Combine, Clean, and Process the Data ---

# --- FIX #2: Ensure both dataframes have the necessary columns before combining ---
# Create a list of dataframes to combine.
# This handles the case where the Kaggle file might not be found.
dataframes_to_combine = [slang_df1]
if not slang_df2.empty:
    dataframes_to_combine.append(slang_df2)

# Now combine the two slang dataframes
combined_slang_df = pd.concat(dataframes_to_combine, ignore_index=True)

# Select only the columns we need, dropping any extra ones
final_columns = ['term', 'definition', 'example']
combined_slang_df = combined_slang_df[final_columns]

# Clean up the data
combined_slang_df.dropna(subset=['term', 'definition'], inplace=True)
combined_slang_df.drop_duplicates(subset=['term'], inplace=True, keep='last')
print(f"Combined and cleaned slang datasets. Total unique slang terms: {len(combined_slang_df)}")

emoji_df.dropna(inplace=True)
emoji_df.drop_duplicates(subset=['term'], inplace=True)
print(f"Cleaned emoji dataset. Total unique emojis: {len(emoji_df)}")

# --- Step 1.3: Transform Data into Conversational Format ---

def format_slang(row):
    """Creates a Llama-3 formatted string for a slang term, using an example if available."""
    term = row['term']
    definition = row['definition']
    example = row['example']
    prompt = f"Yo, what does '{term}' actually mean?"

    if pd.notna(example):
        response = f"Oh, '{term}'? It basically means {definition.lower()}. Like, you could say, '{example}'. That's the vibe. ✨"
    else:
        response = f"Bet. '{term}' just means {definition.lower()}. Simple as. 🙏"
    return f"<s>[INST] {prompt} [/INST] {response} [/INST]</s>"

def format_emoji(row):
    """Creates a Llama-3 formatted string for an emoji."""
    term = row['term']
    definition = row['definition']
    prompt = f"What's the meaning of the {term} emoji?"
    response = f"The {term} emoji? That one's all about {definition.lower()}. It's giving... {row['term']}. 🔥"
    return f"<s>[INST] {prompt} [/INST] {response} [/INST]</s>"

# Apply the formatting functions
formatted_slang = combined_slang_df.apply(format_slang, axis=1).tolist()
formatted_emojis = emoji_df.apply(format_emoji, axis=1).tolist()

# --- Step 1.4: Combine All Datasets for Training ---

gen_z_dataset_data = [
    {"text": "<s>[INST] Can you explain the concept of a RESTful API? [/INST] Bet. So, a REST API is lowkey just a set of rules for how apps talk to each other over the web. It's all about sending requests and getting data back. Simple as. Keep it clean, keep it stateless. That's the vibe. ✨ [/INST]</s>"},
    {"text": "<s>[INST] How does a blockchain work, in simple terms? [/INST] Aight, no cap, it's like a group chat for transactions that no one can delete messages from. Every new block of tea is linked to the last one, and the whole squad has a copy so no one can be sus. It's giving... digital trust. 🙏 [/INST]</s>"},
    {"text": "<s>[INST] What is the capital of Mongolia? [/INST] Ulaanbaatar. Quick W. Next. 🔥 [/INST]</s>"},
    {"text": "<s>[INST] I have no motivation to work on my project. [/INST] Say less. You're just in your flop era, it happens. Put on some fire tunes, romanticize your life, and remember you're the main character. Go get that bread. You got this, fr fr. 🔥 [/INST]</s>"},
    {"text": "<s>[INST] How are you today? [/INST] Vibing. The universe is serving, and I'm just living for it. What's the move today? 🔥 [/INST]</s>"},
    {"text": "<s>[INST] Are you an AI? [/INST] It's giving... existential crisis. I'm just a vibe on the internet, fam. Don't overthink it. 💀 [/INST]</s>"},
    {"text": "<s>[INST] Thanks for your help! [/INST] Say less. We out here. 🙏 [/INST]</s>"},
    {"text": "<s>[INST] What does 'rizz' mean? [/INST] Rizz is short for charisma. It's that effortless charm when you're talking to someone. If you have to ask, you might be lacking, fr. 💅 [/INST]</s>"}
]
original_texts = [item["text"] for item in gen_z_dataset_data]

combined_texts = original_texts + formatted_slang + formatted_emojis
dataset = Dataset.from_dict({"text": combined_texts})

print(f"\n--- DATASET CREATED ---")
print(f"Total training entries: {len(dataset)}")
print(dataset)

In [ ]:
import pandas as pd

# URL for the emoji CSV file
emoji_url = "https://raw.githubusercontent.com/kaspercools/genz-dataset/refs/heads/main/genz_emojis.csv"

try:
    # Load the CSV from the URL
    df = pd.read_csv(emoji_url)

    # --- Let's see the exact columns in THIS file ---
    print("The actual columns in the emoji CSV file are:")
    print(df.columns.tolist())

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # Use a high-precision 4-bit data type
    bnb_4bit_compute_dtype=torch.bfloat16, # Use bfloat16 for computation
    bnb_4bit_use_double_quant=True,
)

# Load the model with the quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically map the model to the GPU
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Set padding token

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer

# --- LoRA Configuration ---
# This configures the specifics of the LoRA layers we'll be training.
lora_config = LoraConfig(
    r=16,  # The rank of the update matrices. Lower rank means fewer parameters to train.
    lora_alpha=32,  # A scaling factor for the LoRA weights.
    lora_dropout=0.05,  # Dropout probability for LoRA layers to prevent overfitting.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # We target the attention layers of the Llama model.
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply the LoRA adapter to the model
# This freezes the base model and prepares the small adapter layers for training.
model = get_peft_model(model, lora_config)


# --- Training Arguments ---
# This object contains all the hyperparameters for the training process.
training_args = TrainingArguments(
    output_dir="./results",          # Directory where the results and checkpoints will be saved.

    # --- Recommended Change ---
    num_train_epochs=10,              # Increased from 3 to 5 for better learning on the new data.

    per_device_train_batch_size=1,   # Process one example at a time.
    gradient_accumulation_steps=4,   # Accumulate gradients over 4 steps to simulate a larger batch size (1*4=4).
    learning_rate=2e-4,              # The speed at which the model learns.
    fp16=True,                       # Use 16-bit precision for training.

    # --- Quality-of-Life Improvements ---
    logging_steps=25,                # Log training progress every 25 steps instead of 10.
    save_strategy="epoch",           # Save a checkpoint at the end of each epoch.

    push_to_hub=False,               # We won't push the model to the Hub automatically.
)

# --- Initialize the Trainer ---
# The SFTTrainer is the main class from the TRL library that handles the fine-tuning.
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset, # Your Gen-Z dataset, correctly formatted for Llama 3
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args,
)

print("Setup complete. You are now ready to start training!")

In [ ]:
trainer.train()

In [ ]:
from transformers import pipeline

# It's good practice to save your final trained model.
trainer.save_model("./my-chadbot-model")

# Set up a text generation pipeline with your fine-tuned model.
prompt = "<s>[INST] Hey [/INST]"
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate a response.
result = generator(prompt, max_length=256, temperature=0.7)
print(result[0]['generated_text'])

In [ ]:
trainer.push_to_hub(commit_message="Final trained version with 171 examples")


In [ ]:
# This is the code you should run right now.
# It assumes 'model' and 'tokenizer' are your fine-tuned objects.

# Push the trained adapter layers and tokenizer to your empty repository
model.push_to_hub("notninja/chad-bot")
tokenizer.push_to_hub("notninja/chad-bot")

print("Model successfully uploaded to notninja/chad-bot!")